In [1]:
import numpy as np
import time
import math
from datetime import datetime
from datetime import timedelta
import pandas as pd
import requests
import json
import pymongo
import itertools

In [2]:
from IPython.display import clear_output

In [3]:
from matplotlib import pyplot as plt
from matplotlib.ticker import MaxNLocator

In [4]:
import pythainlp
from pythainlp import Tokenizer
from pythainlp.util import normalize
from pythainlp.tokenize import word_tokenize
from pythainlp.corpus.common import thai_words
from pythainlp import sent_tokenize
from pythainlp.corpus import thai_stopwords

In [5]:
def create_ListofDict_InsertOne(df):
    info = {
            "Category": df.Category,
            "timestamp": df.timestamp,
            "URL":df.URL,
            "headline": df.headline,
            "newsdate": df.newsdate,
            "newsdate_date": str(df.newsdate_date),
            "totalView": int(df.totalView),
            "totalShare": int(df.totalShare),
            "share_fb": int(df.share_fb),
            "share_tw": int(df.share_tw),
            "share_gp": int(df.share_gp),
            "text": df.text,
            "hl_sentiment_score": float(df.hl_sentiment_score),
            "hl_polarity": df.hl_polarity,
            "hl_neg_kw": df.hl_neg_kw,
            "hl_pos_kw": df.hl_pos_kw,
            "t_sentiment_score": float(df.t_sentiment_score),
            "t_polarity": df.t_polarity,
            "t_neg_kw": df.t_neg_kw,
            "t_pos_kw": df.t_pos_kw,
            "dateDiff": float(df.dateDiff), 
            "avg_share_per_day": float(df.avg_share_per_day),
            "avg_view_per_day": float(df.avg_view_per_day)
    }
    return info

In [6]:
postNum = 30
postNumSleep = math.ceil(60/postNum)

url = "https://api.aiforthai.in.th/ssense"
headers = {
    'Apikey': "ymYbYiJ89ha609Nnv7wuV1CWJFLyCEYo"
    }

In [7]:
def plotSentiment_Dist(op, polarity,topic):
    t_cat_ratio = op.groupby([polarity,'Category']).agg({topic: lambda x: len(x)}).reset_index('Category')
    #d_ax1 = t_cat_ratio[t_cat_ratio.Category=='Crime'][topic]
    d_ax2 = t_cat_ratio[t_cat_ratio.Category=='Politic'][topic]
    #d_ax3 = t_cat_ratio[t_cat_ratio.Category=='Entertain'][topic]

    #bar_label1,bar_label2,bar_label3 = d_ax1.index,d_ax2.index,d_ax3.index
    bar_label2 = d_ax2.index

    #fig, ax = plt.subplots(1, 3, figsize=(15, 5), sharey=True)
    fig, ax = plt.subplots(1, 1, figsize=(15, 5), sharey=True)
    #ax[0].grid(color='k', linestyle='-', linewidth=0.2)
    #ax[0].bar(bar_label1, d_ax1)
    #ax[0].set_xticklabels(d_ax1.index)
    #ax[0].set_title(f'Sentiment distribution : {topic} - Crime')
    #ax[0].set_xlabel('Sentiment')
    #ax[0].set_ylabel('count_news')

    ax.grid(color='k', linestyle='-', linewidth=0.2)
    ax.bar(bar_label2, d_ax2)
    ax.set_xticklabels(d_ax2.index)
    ax.set_title(f'Sentiment distribution : {topic} - Politic')
    ax.set_xlabel('Sentiment')
    ax.set_ylabel('count_news')

    #ax[2].grid(color='k', linestyle='-', linewidth=0.2)
    #ax[2].bar(bar_label3, d_ax3)
    #ax[2].set_xticklabels(d_ax3.index)
    #ax[2].set_title(f'Sentiment distribution : {topic} - Entertain')
    #ax[2].set_xlabel('Sentiment')
    #ax[2].set_ylabel('count_news')

    plt.show()

### Mark sentiment

In [8]:
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["BADS7101"]
#q_c_urlList = mydb["URL_LIST_CRIME"]
#i_c_scrape = mydb["CRIME_NEWS"]

q_p_urlList = mydb["URL_LIST_POLITIC_REMDUP"]
i_p_scrape = mydb["POLITIC_NEWS_U"]

#q_e_urlList = mydb["URL_LIST_ENTER"]
#i_e_scrape = mydb["ENTERTAIN_NEWS"]

In [9]:
cursor_pol = q_p_urlList.find()
cursor_pol_s = i_p_scrape.find()

df_pol_url = pd.DataFrame(cursor_pol)
df_pol_detail = pd.DataFrame(cursor_pol_s)
df_pol_join = pd.merge(df_pol_url,df_pol_detail,how="inner",on=['URL'])
df_pol_join.insert(0,'Category',value='Politic')

In [10]:
#df_plotViz = pd.concat([df_crime_join,df_pol_join,df_ent_join],ignore_index=True)
df_plotViz = df_pol_join.copy()

In [11]:
df_plotViz.insert(6,'newsdate_date',value=None)
df_plotViz['newsdate_date'] = df_plotViz['newsdate'].dt.date

In [12]:
df_plotViz.shape

(5522, 14)

In [13]:
df_plotViz.drop(['_id_x', '_id_y'], axis=1,inplace=True)

In [14]:
mind = datetime.strptime(str('2020-07-01 00:00:00'), '%Y-%m-%d %H:%M:%S')
maxd = datetime.strptime(str('2020-10-24 23:59:00'), '%Y-%m-%d %H:%M:%S')
# เอาเฉพาะ avg_share_per_day สูงกว่า mean หมวดข่าวนั้น
df_plotViz = df_plotViz[(df_plotViz.newsdate>=mind)&(df_plotViz.newsdate<maxd)]
df_plotViz.shape

(4961, 12)

In [15]:
current_day = datetime.strptime(str('2020-10-25 23:59:00'), '%Y-%m-%d %H:%M:%S')
df_plotViz['dateDiff']  = df_plotViz['newsdate'].apply(lambda x : current_day - x )
df_plotViz['dateDiff']  = df_plotViz['dateDiff'].apply(lambda x : x.days)
df_plotViz['avg_share_per_day']  = df_plotViz['totalShare'] / df_plotViz['dateDiff']
df_plotViz['avg_view_per_day']  = df_plotViz['totalView'] / df_plotViz['dateDiff']

In [16]:
df_plotViz.insert(12,'hl_sentiment_score',value=None)
df_plotViz.insert(13,'hl_polarity',value=None)
df_plotViz.insert(14,'hl_neg_kw',value=None)
df_plotViz.insert(15,'hl_pos_kw',value=None)
df_plotViz.insert(16,'t_sentiment_score',value=None)
df_plotViz.insert(17,'t_polarity',value=None)
df_plotViz.insert(18,'t_neg_kw',value=None)
df_plotViz.insert(19,'t_pos_kw',value=None)

In [17]:
df_plotViz.tail(2)

,Category,timestamp,URL,headline,newsdate,newsdate_date,totalView,totalShare,share_fb,share_tw,...,hl_polarity,hl_neg_kw,hl_pos_kw,t_sentiment_score,t_polarity,t_neg_kw,t_pos_kw,dateDiff,avg_share_per_day,avg_view_per_day
5520,Politic,2020-11-09 18:05:03.083,https://www.dailynews.co.th/politics/788738,เตือน!คนรุ่นใหม่ลุกฮือสู้เผด็จการ ล่า5หมื่นชื่...,2020-08-07 15:21:00,2020-08-07,1675,10.0,5.0,2,...,None,None,None,None,None,None,None,79,0.126582,21.202532
5521,Politic,2020-11-09 18:05:03.083,https://www.dailynews.co.th/politics/788726,"""ชวน""เตือนสังคมไทยอย่าเกรงใจจนทำสิ่งไม่ถูกต้อง",2020-08-07 15:16:00,2020-08-07,1382,15.0,8.0,3,...,None,None,None,None,None,None,None,79,0.189873,17.493671


In [18]:
df_plotViz.describe()

,totalView,totalShare,share_fb,share_tw,share_gp,dateDiff,avg_share_per_day,avg_view_per_day
count,4961.000000,4961.000000,4961.000000,4961.000000,4961.000000,4961.000000,4961.000000,4961.000000
mean,3689.912114,26.361419,14.799234,5.603709,5.954042,55.367466,1.570616,230.663748
std,10665.259922,127.684666,125.373970,8.762099,6.338622,33.837048,30.810389,1382.630855
min,233.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.026316,3.979592
25%,1237.000000,11.000000,4.000000,3.000000,4.000000,26.000000,0.182609,20.765625
50%,1697.000000,14.000000,5.000000,4.000000,5.000000,55.000000,0.297872,40.073171
75%,3015.000000,19.000000,7.000000,6.000000,6.000000,84.000000,0.636364,100.428571
max,313770.000000,3010.000000,3000.000000,218.000000,146.000000,116.000000,2140.000000,49658.000000


In [19]:
# เอาเฉพาะ avg_view_per_day สูงกว่า mean หมวดข่าวนั้น
df_plotViz = df_plotViz[(df_plotViz.avg_share_per_day>=1.570616)]
df_plotViz.shape

(549, 23)

In [20]:
df_plotViz.describe()

,totalView,totalShare,share_fb,share_tw,share_gp,dateDiff,avg_share_per_day,avg_view_per_day
count,549.000000,549.000000,549.00000,549.000000,549.000000,549.000000,549.000000,549.000000
mean,11976.202186,107.806922,85.80510,12.471767,9.489982,11.480874,11.174521,1591.840062
std,29529.109441,373.389641,369.39973,23.769683,16.448171,19.412589,92.127040,3894.890145
min,540.000000,7.000000,2.00000,2.000000,1.000000,1.000000,1.571429,11.989796
25%,1273.000000,12.000000,5.00000,3.000000,3.000000,4.000000,2.125000,305.600000
50%,3017.000000,17.000000,6.00000,5.000000,4.000000,6.000000,3.250000,590.750000
75%,7747.000000,31.000000,12.00000,9.000000,7.000000,9.000000,6.400000,1246.000000
max,313770.000000,3010.000000,3000.00000,218.000000,146.000000,113.000000,2140.000000,49658.000000


In [21]:
sentiment_db = mydb["SENTIMENT_DAILYNEWS_API"]
sentiment_db.delete_many({})

In [22]:
error_elem_h,error_elem_t = [],[]
apilist_header = []

for i,e in enumerate(df_plotViz.URL,0):
    print('Round:',i)
    df_process = df_plotViz.iloc[i].copy().fillna(0)
    h_data,t_data = {'text':df_process.headline.replace('%', '').encode('utf-8')},{'text':df_process.text.replace('%', '').encode('utf-8')}
    try:
        h_response = requests.post(url, data=h_data, headers=headers)
        h_score = h_response.json()['sentiment']['score']
        h_polarity = h_response.json()['sentiment']['polarity']
        h_neg_kw = h_response.json()['preprocess']['neg']
        h_pos_kw = h_response.json()['preprocess']['pos']
        df_process.hl_sentiment_score,df_process.hl_polarity,df_process.hl_neg_kw,df_process.hl_pos_kw = h_score, h_polarity, h_neg_kw, h_pos_kw
        #time.sleep(postNumSleep)
    except:
        error_elem_h.append(i)
    ####
    try:
        t_response = requests.post(url, data=t_data, headers=headers)
        t_score = t_response.json()['sentiment']['score']
        t_polarity = t_response.json()['sentiment']['polarity']
        t_neg_kw = t_response.json()['preprocess']['neg']
        t_pos_kw = t_response.json()['preprocess']['pos']
        df_process.t_sentiment_score,df_process.t_polarity,df_process.t_neg_kw,df_process.t_pos_kw = t_score, t_polarity, t_neg_kw, t_pos_kw
        #time.sleep(postNumSleep)
    except:
        error_elem_t.append(i)
    process_data = create_ListofDict_InsertOne(df_process)
    print(process_data)
    sentiment_db.insert_one(process_data)
    time.sleep(postNumSleep)
    clear_output(wait=True)

Round: 548
{'Category': 'Politic', 'timestamp': Timestamp('2020-11-09 18:03:41.276000'), 'URL': 'https://www.dailynews.co.th/politics/789028', 'headline': "'นันทเดช'ปลุก'กปปส.'คืนชีพ เป่านกหวีดชุมนุม10ส.ค.", 'newsdate': Timestamp('2020-08-09 14:20:00'), 'newsdate_date': '2020-08-09', 'totalView': 171600, 'totalShare': 425, 'share_fb': 142, 'share_tw': 164, 'share_gp': 119, 'text': 'เมื่อวันที่ 9 ส.ค.เพจเฟซบุ๊ก พล.ท.นันทเดช เมฆสวัสดิ์ อดีตกรรมการบริหารพรรครวมพลังประชาชาติไทย (รปช.) อดีตแนวร่วม กปปส. และอดีตหัวหน้าศูนย์ปฏิบัติการพิเศษ ศูนย์รักษาความปลอดภัยแห่งชาติ (ศรภ.) ได้โพสต์ข้อความลงบนเฟซบุ๊กส่วนตัว ระบุว่า "ขอเชิญทุกอาชีพ มากันเอง ออกตังค์เอง แบบเดิมๆ ที่หน้ารัฐสภาใหม่ครับ ทำกิจกรรมสันติวิธี วันจันทร์ที่ 10 เวลา 10.10 น."\n\n\n\nนอกจากนี้ พล.ท.นันทเดช ยังโพสต์ภาพพร้อมข้อความ ของ ศูนย์กลางประสานนักศึกษา อาชีวะ ประชาชนปกป้องสถาบันฯ หรือ ศอปส. ที่ระบุว่า 10 สิงหาคม ขอเชิญชวนพ่อแม่พี่น้องและเยาวชนทุกท่าน ร่วมกันปกป้อง ชาติ ศาสนา พระมหากษัตริย์ เพื่อแสดงจุดยืนต่อต้านแกนนำที่ปราศรัยหมิ่น

In [23]:
# Element รันมือเพื่อตามเก็บ
print('Error in h:',error_elem_h,'Error in t',error_elem_t)

Error in h: [] Error in t []


### 